In [2]:
from classiq import FunctionGenerator, RegisterUserInput, QUInt, FunctionLibrary, Model, show, synthesize
from classiq.builtin_functions import Adder, Negation, StatePreparation

In [4]:
negation_params = Negation(arg=RegisterUserInput(size=3), inplace=True)
adder_params = Adder(
     left_arg=RegisterUserInput(size=3), right_arg=negation_params.outputs[negation_params.output_name]
)

In [5]:
function_generator = FunctionGenerator(function_name="my_subtractor_function")

input_dictionary = function_generator.create_inputs(
    {"input_arg_1": QUInt[3], "input_arg_2": QUInt[3]}
)

negation_dictionary = function_generator.Negation(
    negation_params, in_wires={"arg": input_dictionary["input_arg_2"]}
)
adder_outputs = function_generator.Adder(
    adder_params,
    in_wires={
        "left_arg": input_dictionary["input_arg_1"],
        "right_arg": negation_dictionary[negation_params.output_name]
    },
)

function_generator.set_outputs({"difference": adder_outputs[adder_params.output_name]})

In [6]:
composite_function = function_generator.to_function_definition()
function_library = FunctionLibrary(composite_function)

probabilities = (0.5, 0.1, 0.2, 0.005, 0.015, 0.12, 0.035, 0.025)
sp_params = StatePreparation(probabilities=probabilities)

model = Model()
model.include_library(function_library)
sp_outputs = model.StatePreparation(sp_params)

model.my_subtractor_function(in_wires={"input_arg_1": sp_outputs["OUT"]})

quantum_program = synthesize(model.get_model())
show(quantum_program)

Opening: https://platform.classiq.io/circuit/3b6a429c-e321-4468-8889-5d4df09aa199?version=0.34.0
